In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    """
    Takes a single wikipedia record, extracts all known values for alternate titles,
    and moves them to a list.  Additionally maps redundant/duplicative column names.
    """
    movie = dict(movie) # creates a non-destructive copy
    
    # Clean alternate titles
    alt_titles = dict()
    languages = ['Arabic',
                 'Cantonese',
                 'Chinese',
                 'French',
                 'Hangul',
                 'Hebrew',
                 'Hepburn',
                 'Japanese',
                 'Literally',
                 'Mandarin',
                 'McCune–Reischauer',
                 'Polish',
                 'Revised Romanization',
                 'Romanized',
                 'Russian',
                 'Simplified',
                 'Traditional',
                 'Yiddish']

    for language in languages:
        if language in movie:
            alt_titles[language] = movie[language]
            movie.pop(language)

    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    

    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director(s)')
    change_column_name('Director', 'Director(s)')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Written by', 'Writer(s)')
    change_column_name('Original release', 'Release date')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Music by', 'Composer(s)')
    
    return movie

In [3]:
def parse_dollars(s):
    """
    Given string s, parse currency strings to float.
    """
    if type(s) != str:
        return np.nan
    
    # form one: r"\$\s*\d{1,3}\.?\d*\s*[mb]illi?on"
    # form two: r"\$\s*\d+[,\.]\d{3}"
    
    # form: "$###.# billion: 
    # remove dollar signs, whitespace, and text.  
    # Multiply by 1billion
    if re.match(r"\$\s*\d{1,3}\.?\d*\s*billi?on", s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]', '', s)
        value = float(s) * 10**9
        return value
     
    # form: "$###.# million: 
    # remove dollar signs, whitespace, and text.  
    # Multiply by 1million   
    if re.match(r"\$\s*\d{1,3}\.?\d*\s*milli?on", s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]', '', s)
        value = float(s) * 10**6
        return value    
    
    # form: $###,###,###
    # strip dollar signs and thousands separators
    if re.match(r"\$\s*\d+[,\.]\d{3}", s, flags=re.IGNORECASE):
        s = re.sub('\$|,|\.','',s)
        value = float(s)
        return value
        
    else:
        return np.nan

In [4]:
def import_source_files(wiki_file: str,
                        kaggle_file: str,
                        ratings_file: str):
    """
    Function takes three arguments, each corresponding to the name
    of a specific source csv or json file for the three types of data 
    objects we are importing.  Returns all three objects as unique
    pandas DataFrames.
    """

    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)

    with open(wiki_file, mode='r') as file:
        wiki_movies_json = json.load(file)
    
    # Remove TV shows
    wiki_movies_json = [wiki_movies_json[i]\
                        for i in range(len(wiki_movies_json))\
                        if 'No. of episodes' not in wiki_movies_json[i]]
    
    # Iterate through clean movie function to tidy columns
    wiki_movies_json = [clean_movie(wiki_movies_json[i]) for i in range(len(wiki_movies_json))]
    
    # Create dataframe
    wiki_movies_df = pd.DataFrame(wiki_movies_json)
    
    # Extract all IMDB IDs from valid URls and remove records that do not contain them
    try:
        wiki_movies_df.dropna(subset=['imdb_link'], inplace=True)
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e:
        print(f'IMDB extraction failed.  {e}')
    
    #Consolidate writer columns without overwriting
    for col in ["Writer(s)", "Screenplay by", "Story by"]:
        wiki_movies_df[col] = wiki_movies_df[col].apply(lambda x: ', '.join(x) if type(x) == list else x)
    
    wiki_movies_df["Writer(s)"] = wiki_movies_df.apply(lambda row: row["Screenplay by"] if pd.isna(row["Writer(s)"]) else row["Writer(s)"], axis=1)
    wiki_movies_df["Writer(s)"] = wiki_movies_df.apply(lambda row: row["Story by"] if pd.isna(row["Writer(s)"]) else row["Writer(s)"], axis=1)

    wiki_movies_df.drop(columns=["Screenplay by", "Story by"], inplace=True)
    
    columns_to_drop = [column\
                         for column in wiki_movies_df.columns\
                         if wiki_movies_df[column].count()/len(wiki_movies_df) <= 0.1]   
        
    wiki_movies_df.drop(columns=columns_to_drop, inplace=True)
    
    #Convert year to int
    wiki_movies_df["year"] = wiki_movies_df["year"].apply(lambda x: int(x))
    
    #Regex strings for currency patterns                                   
    form_one = r"\$\s*\d{1,3}\.?\d*\s*[mb]illi?on"                                       
    form_two = r"\$\s*\d+[,\.]\d{3}"
    
    # CLEAN BOX OFFICE DATA
    box_office = wiki_movies_df['Box office'].dropna()
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)                                       
                                       
    wiki_movies_df['box_office'] = box_office.str.\
                                    extract(f"({form_one}|{form_two})",\
                                            flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    # CLEAN BUDGET data
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    
    # Omit wikipedia citation markers using square brackets
    budget = budget.str.replace(r'\[\d+\]\s*', '')

    # Remove any hyphens and defer to smaller end of range
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])' , '$', regex=True)

    contains_form_one = budget.str.contains(pat=form_one, flags=re.IGNORECASE, na=False)
    contains_form_two = budget.str.contains(pat=form_two, flags=re.IGNORECASE, na=False)
    
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Budget', axis=1, inplace=True)
    
    # CLEAN RELEASE DATE DATA
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # pattern 1: Month Name, 1-2 digits, 4 digit year
    date_pat_1 = r"\w*\s\d{1,2},\s\d{4}"
    matches_pat_1 = release_date.str.contains(date_pat_1, flags=re.IGNORECASE, na=False)

    # pattern 2: yyyy-dd-mm
    date_pat_2 = r"\d{4}[-—–]\d{2}[-—–]\d{2}"
    matches_pat_2 = release_date.str.contains(date_pat_2, flags=re.IGNORECASE, na=False)

    # pattern 3: (optional day), month name, year
    date_pat_3 = r"\d{0,2}\s*\w{3,10}\s\d{4}"
    matches_pat_3 = release_date.str.contains(date_pat_3, flags=re.IGNORECASE, na=False)

    # pattern 4: four digit year only
    date_pat_4 = r"\d{4}"
    matches_pat_4 = release_date.str.contains(date_pat_4, flags=re.IGNORECASE, na=False)

    wiki_movies_df['release_date'] = pd.to_datetime(
        release_date.str.extract(f'({date_pat_1}|{date_pat_2}|{date_pat_3}|{date_pat_4})')[0],
        infer_datetime_format=True,
        errors='coerce')
    
    wiki_movies_df.drop('Release date', axis=1, inplace=True)    
    
    # RUNTIME DATA
    # two string forms transformed: "# h(ours) ## m(inutes)", and '### minutes"
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r"(\d+)\s*ho?u?r?s?\s*(\d*)|(\d{1,3})\s*m")
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)

     # 2. Clean the Kaggle metadata.
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult', axis='columns')
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise') 

    # 3. Merged the two DataFrames into the movies DataFrame.
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])

    # 4. Drop unnecessary columns from the merged DataFrame.
    movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index)
    movies_df.drop(columns=['title_wiki', 'release_date_wiki', 'Language', 'Production company(s)'], inplace=True)
    
    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
         lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
            , axis = 1)
        df.drop(columns=wiki_column, inplace=True)                                                           

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(movies_df, kaggle_column='runtime', wiki_column='running_time')
    fill_missing_kaggle_data(movies_df, kaggle_column='budget_kaggle', wiki_column='budget_wiki')
    fill_missing_kaggle_data(movies_df, kaggle_column='revenue', wiki_column='box_office')

    # 7. Filter the movies DataFrame for specific columns.
    movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                           'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                           'genres','original_language','overview','spoken_languages','Country',
                           'production_companies','production_countries','Distributor',
                           'Producer(s)','Starring','Cinematography','Editor(s)','Writer(s)','Based on'
                          ]]

    # 8. Rename the columns in the movies DataFrame.
    movies_df.rename({'id':'kaggle_id',
                      'title_kaggle':'title',
                      'url':'wikipedia_url',
                      'budget_kaggle':'budget',
                      'release_date_kaggle':'release_date',
                      'Country':'country',
                      'Distributor':'distributor',
                      'Producer(s)':'producers',
                      'Director':'director',
                      'Starring':'starring',
                      'Cinematography':'cinematography',
                      'Editor(s)':'editors',
                      'Writer(s)':'writers',
                      'Composer(s)':'composers',
                      'Based on':'based_on'
                     }, axis='columns', inplace=True)

    # 9. Transform and merge the ratings DataFrame
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                        .rename({'userId':'count'}, axis=1) \
                        .pivot(index='movieId', columns='rating', values='count')
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
    
    return wiki_movies_df, movies_df, movies_with_ratings_df

In [5]:
file_dir = './data'
wiki_file = f'{file_dir}/wikipedia-movies.json'
kaggle_file = f'{file_dir}/movies_metadata.csv'
ratings_file = f'{file_dir}/ratings.csv'

wiki_file, kaggle_file, ratings_file = import_source_files(wiki_file=wiki_file, 
                                                           kaggle_file=kaggle_file, 
                                                           ratings_file=ratings_file)

C:\Users\espoe\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:72: FutureWarning: The default value of regex will change from True to False in a future version.


In [6]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_df = kaggle_file
movies_with_ratings_df = ratings_file

In [7]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Country,Language,Director(s),...,Editor(s),Producer(s),Production company(s),Composer(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,United States,English,Renny Harlin,...,Michael Tronick,"[Steve Perry, Joel Silver]",Silver Pictures,"[Cliff Eidelman, Yello]","David Arnott, James Cappe, Daniel Waters",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,United States,English,James Foley,...,Howard E. Smith,"[Ric Kidney, Robert Redlin]",Avenue Pictures,Maurice Jarre,"James Foley, Robert Redlin",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,United States,"[English, Lao]",Roger Spottiswoode,...,"[John Bloom, Lois Freeman-Fox]",Daniel Melnick,"[Carolco Pictures, IndieProd Company]",Charles Gross,"John Eskow, Richard Rush",tt0099005,57718.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,United States,English,Woody Allen,...,Susan E. Morse,Robert Greenhut,NaN,NaN,Woody Allen,tt0099012,7331.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,US,English,John Cornell,...,David Stiven,John Cornell,NaN,Maurice Jarre,Paul Hogan,tt0099018,6939.0,25000000.0,1990-12-19,95.0


In [8]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,rating_0.5,rating_1.0,rating_1.5,rating_2.0,rating_2.5,rating_3.0,rating_3.5,rating_4.0,rating_4.5,rating_5.0
0,tt0098987,9548,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane,Kojak. Columbo. Dirty Harry. Wimps.,NaN,https://en.wikipedia.org/wiki/The_Adventures_o...,https://www.imdb.com/title/tt0098987/,104.0,49000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,tt0098994,25501,"After Dark, My Sweet","After Dark, My Sweet",All they risked was everything.,NaN,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",https://www.imdb.com/title/tt0098994/,114.0,6000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,tt0099005,11856,Air America,Air America,The few. The proud. The totally insane.,NaN,https://en.wikipedia.org/wiki/Air_America_(film),https://www.imdb.com/title/tt0099005/,112.0,35000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,tt0099012,8217,Alice,Alice,NaN,NaN,https://en.wikipedia.org/wiki/Alice_(1990_film),https://www.imdb.com/title/tt0099012/,102.0,12000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he think he is?,NaN,https://en.wikipedia.org/wiki/Almost_an_Angel,https://www.imdb.com/title/tt0099018/,95.0,25000000.0,...,3.0,0.0,3.0,2.0,5.0,26.0,37.0,46.0,16.0,11.0


In [9]:
# 15. Check the movies_df DataFrame. 
movies_df.head()

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,country,production_companies,production_countries,distributor,producers,starring,cinematography,editors,writers,based_on
0,tt0098987,9548,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane,Kojak. Columbo. Dirty Harry. Wimps.,NaN,https://en.wikipedia.org/wiki/The_Adventures_o...,https://www.imdb.com/title/tt0098987/,104.0,49000000.0,...,United States,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",20th Century Fox,"[Steve Perry, Joel Silver]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,Michael Tronick,"David Arnott, James Cappe, Daniel Waters","[Characters, by Rex Weiner]"
1,tt0098994,25501,"After Dark, My Sweet","After Dark, My Sweet",All they risked was everything.,NaN,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",https://www.imdb.com/title/tt0098994/,114.0,6000000.0,...,United States,"[{'name': 'Avenue Pictures Productions', 'id':...","[{'iso_3166_1': 'US', 'name': 'United States o...",Avenue Pictures,"[Ric Kidney, Robert Redlin]","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,Howard E. Smith,"James Foley, Robert Redlin","[the novel, After Dark, My Sweet, by, Jim Thom..."
2,tt0099005,11856,Air America,Air America,The few. The proud. The totally insane.,NaN,https://en.wikipedia.org/wiki/Air_America_(film),https://www.imdb.com/title/tt0099005/,112.0,35000000.0,...,United States,"[{'name': 'IndieProd Company Productions', 'id...","[{'iso_3166_1': 'US', 'name': 'United States o...",TriStar Pictures,Daniel Melnick,"[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[John Bloom, Lois Freeman-Fox]","John Eskow, Richard Rush","[Air America, by, Christopher Robbins]"
3,tt0099012,8217,Alice,Alice,NaN,NaN,https://en.wikipedia.org/wiki/Alice_(1990_film),https://www.imdb.com/title/tt0099012/,102.0,12000000.0,...,United States,"[{'name': 'Orion Pictures', 'id': 41}]","[{'iso_3166_1': 'US', 'name': 'United States o...",Orion Pictures,Robert Greenhut,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,Susan E. Morse,Woody Allen,NaN
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he think he is?,NaN,https://en.wikipedia.org/wiki/Almost_an_Angel,https://www.imdb.com/title/tt0099018/,95.0,25000000.0,...,US,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'US', 'name': 'United States o...",Paramount Pictures,John Cornell,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,David Stiven,Paul Hogan,NaN
